In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from sqlalchemy import create_engine, inspect

In [2]:
from secret import DESTINATION_DB_URL

In [3]:
# Check the tables that are available in destination_db
engine = create_engine(DESTINATION_DB_URL)
inspector = inspect(engine)
inspector.get_table_names()

['customers',
 'orders',
 'orderlines',
 'products',
 'stabcensus',
 'calendar',
 'campaigns']

In [8]:
# Test query and see if the data is loaded correctly, and see if the memory usage make sense
query = """
select * from orders
limit 5
"""
df = pd.read_sql(query, engine)
display(df)
df.info()

,orderid,numorderlines,numunits,paymenttype,totalprice,state,city,orderdate,campaignid
0,1002854,3,3,VI,$190.00,MA,NEWTON,2009-10-13,2141
1,1002855,1,1,VI,$10.00,NY,NEW ROCHELLE,2009-10-13,2173
2,1002856,2,2,AE,$35.22,FL,MIAMI,2011-06-02,2141
3,1002857,1,1,AE,$10.00,NJ,E RUTHERFORD,2009-10-14,2173
4,1002886,1,1,VI,$10.00,MD,BALTIMORE,2010-11-19,2141


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   orderid        5 non-null      int64 
 1   numorderlines  5 non-null      int64 
 2   numunits       5 non-null      int64 
 3   paymenttype    5 non-null      object
 4   totalprice     5 non-null      object
 5   state          5 non-null      object
 6   city           5 non-null      object
 7   orderdate      5 non-null      object
 8   campaignid     5 non-null      int64 
dtypes: int64(4), object(5)
memory usage: 488.0+ bytes


### EDA 

In [10]:
# see the unique value in calendar holidaytype
query = """
select 
    holidaytype,
    count(date) as count
from calendar
group by holidaytype
order by count desc;
"""
holiday_type = pd.read_sql(query, engine)
holiday_type

,holidaytype,count
0,,33207
1,minor,909
2,national,909
3,jewish,775
4,other,522
5,christian,293
6,muslim,187
7,chinese,88


In [34]:
# let's see what is the distribution of holiday and the campaign it used
query = """
select
    case when c.holidaytype = '' then 'N' else 'Y' end as isholiday,
    ca.discount,
    ca.freeshppingflag,
    count(o.orderid) as order_count
from orders o
left join calendar c on o.orderdate=c.date
left join campaigns ca on o.campaignid=ca.campaignid
group by 
    isholiday,
    ca.discount,
    ca.freeshppingflag;
"""
df_campaign_holiday = pd.read_sql(query, engine)
df_campaign_holiday

,isholiday,discount,freeshppingflag,order_count
0,N,0,N,100570
1,N,0,Y,902
2,N,10,N,69603
3,N,10,Y,557
4,N,15,N,140
5,N,20,N,161
6,N,20,Y,1049
7,N,25,N,167
8,N,30,N,406
9,N,32,N,16


In [35]:
# let's see if there any association between freeshipping/discount and holiday
contingency_table = pd.crosstab(df_campaign_holiday['isholiday'],df_campaign_holiday['freeshppingflag'], values=df_campaign_holiday['order_count'], aggfunc='sum')
display(contingency_table)
# Perform Chi-Square Test
chi2, p_value, _, _ = stats.chi2_contingency(contingency_table)

print(f"Chi-Square Statistic: {chi2}")
print(f"P-value: {p_value}")

corr_coef = (df_campaign_holiday['isholiday']=='Y').corr(df_campaign_holiday['discount'])
print(f"Correlation coefficient: {corr_coef}")

freeshppingflag,N,Y
isholiday,,
N,171209,2508
Y,18649,617


Chi-Square Statistic: 335.64846751365565
P-value: 5.656850205642618e-75
Correlation coefficient: -0.0927392438793299


small p-value indicates that maybe there is a significant association between holiday and freeshipping, <br>
the correlation coefficient close to zero indicates that the relation between either it is holiday or not and the amount of discount is weak

In [38]:
df_campaign_holiday['isdiscount'] = df_campaign_holiday['discount'].apply(lambda x: 'Y' if x>0 else 'N')
contingency_table = pd.crosstab(df_campaign_holiday['isholiday'],df_campaign_holiday['isdiscount'], values=df_campaign_holiday['order_count'], aggfunc='sum')
display(contingency_table)
# Perform Chi-Square Test
chi2, p_value, _, _ = stats.chi2_contingency(contingency_table)

print(f"Chi-Square Statistic: {chi2}")
print(f"P-value: {p_value}")

isdiscount,N,Y
isholiday,,
N,101472,72245
Y,10917,8349


Chi-Square Statistic: 21.707190149489733
P-value: 3.175986494210284e-06


small p-value indicates that maybe there is a significant association between either it is holiday or not and if there is a discount or not.